# * TB3S000900 : TOL Gross Adds - Connected (Install Location)

## Parameter

In [1]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

In [2]:
config = configparser.ConfigParser()
config.read('../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 13)

## ETL Process

### Step 1 : RAW_TOL_GA_CON_INSTLTN_LOC
    Truncate -> Insert

In [20]:
# Data Source

print(f'\nJob Start...')

with open('SQL/Import-RAW_TOL_GA_CON_INSTLTN_LOC.sql', 'r') as sql_file:
    queries = sql_file.read().split(';')
    query = queries[0].strip()
    sql_file.close()
    
# TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
v_date = (curr_dt, )

try:
    # Create Rawdata
    print(f'\n Processing...')
    src_cur.execute(query)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n -> df : {src_df.shape[0]} rows, {src_df.shape[1]} columns, data as of {v_date[0]}')
    # print(f'\n -> src_df :\n{src_df}')

    # AKPIPRD
    tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    tgt_conn = oracledb.connect(tgt_dsn)
    print(f'\n{AKPIPRD_db} : Connected')
    tgt_cur = tgt_conn.cursor()

    # Truncate
    # tgt_cur.execute("TRUNCATE TABLE AUTOKPI.RAW_TOL_GA_CON_INSTLTN_LOC")
    # print(f'\n -> TRUNCATE : "RAW_TOL_GA_CON_INSTLTN_LOC" : Done !')

    # Insert
    tgt_cur.executemany("INSERT INTO RAW_TOL_GA_CON_INSTLTN_LOC\
                        (INSTALLATION_DT, APPLICATION_DT, LAST_DT, ASSET_NUM, ORDER_TP, DISCONNECT_TP, BAN_NUM, PROMOTION_CODE, SPEED, PRODUCT, CUST_NAME, ID_NUM, EMP_NUM, PRTNR_CD, EMP_NAME, ENTRY_FEE, BUILDING_ID, BUILDING_NO, HOUSING, SCAB_CODE, LATITUDE, LONGITUDE, INSTLLR_CCAATT, ACCT_TYP, ACCT_SUB_TYP, INSTLTN_CCAATT_CD, BILL_CCAATT_CD, MOST_USE_CCAATT_CD, SL_CCAATT_CD, SOC_NM, SOC_EFF_DT, SOC_EXP_DT, PROD_OFR_DSCR, CNTRCT_TERM)\
                        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27,:28,:29,:30,:31,:32,:33,:34)", rows)
    tgt_cur.close()
    tgt_conn.commit()
    print(f'\n -> INSERT : "RAW_TOL_GA_CON_INSTLTN_LOC" : Done !')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    tgt_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')


Job Start...

TDMDBPR : Connected

 Processing...

 -> df : 50426 rows, 34 columns, data as of 2024-06-13

AKPIPRD : Connected

 -> INSERT : "RAW_TOL_GA_CON_INSTLTN_LOC" : Done !

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


In [18]:
src_df.tail()
# src_df.dtypes
# df[['PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates()

,INSTALLATION_DT,APPLICATION_DT,LAST_DT,ASSET_NUM,ORDER_TP,DISCONNECT_TP,BAN_NUM,PROMOTION_CODE,SPEED,PRODUCT,...,ACCT_SUB_TYP,INSTLTN_CCAATT_CD,BILL_CCAATT_CD,MOST_USE_CCAATT_CD,SL_CCAATT_CD,SOC_NM,SOC_EFF_DT,SOC_EXP_DT,PROD_OFR_DSCR,CNTRCT_TERM
22373,2024-06-11 14:20:58,2024-06-08 11:30:45,2024-06-11 14:20:58,9606135003,I,NEW,None,FTOT212-500,500/500,FTTH,...,RES,660108,660108,None,660101,None,NaT,NaT,None,None
22374,2024-06-11 12:21:33,2024-06-10 16:58:32,2024-06-11 12:21:33,9611430490,I,NEW,None,FTOT212-500,500/500,FTTH,...,RES,501408,501407,None,500709,None,NaT,NaT,None,None
22375,2024-06-11 14:11:01,2024-06-07 17:29:02,2024-06-11 14:11:01,9611425381,I,MG-F,None,FTOT177-500,500/500,FTTH,...,RES,840405,840405,None,888888,DC200A,2024-06-11,NaT,TrueOnline Special Discount - TOL Fix Amount,None
22376,2024-06-11 16:09:15,2024-06-08 20:09:54,2024-06-11 16:09:15,9611427762,I,NEW,None,FTOT227-500,500/500,FTTH,...,RES,104703,104703,None,102107,DCFAL0450B01,2024-06-11,NaT,Special Discount 450B (DCFAL0450B01),None
22377,2024-06-11 19:02:45,2024-06-10 13:18:19,2024-06-11 19:02:45,9611429484,I,NEW,None,FTOT135-1000,1024/500,FTTH,...,RES,130106,130106,None,888888,None,NaT,NaT,None,None


### Step 2 : Generate "Sales Performance Report"
    Excel file

In [34]:
# Output to Excel

print(f'\nJob Start...')

query = """
    /*** Sales Performance Report ***/
    -----------------------------------------------------------------------------------------------------------------------

    WITH W_SALE_TARGET_MONTHLY AS 
    (
        SELECT TM_KEY_MTH, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, SUM(METRIC_VALUE) AS TARGET_MTH
        
        FROM AUTOKPI.ADHOC_DTAC_TARGET_AREA_DAILY
        
        WHERE METRIC_GRP = 'Sales'
        AND REGEXP_LIKE(METRIC_CD, 'DB1R000900|DB2R000500|TB1R000900|TB2R000500|TB3R000600|TB4R001000')
        AND REGEXP_LIKE(METRIC_CD, '[0-9]$|[0-9]A[A-K]$')
        
        GROUP BY TM_KEY_MTH, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_ADJUST_TARGET_DTAC AS 
    (
        SELECT A.TM_KEY_YR, A.TM_KEY_QTR, A.TM_KEY_MTH, A.PRODUCT_GRP, A.COMP_CD, A.METRIC_CD, A.METRIC_NAME
            , CASE WHEN REGEXP_LIKE(A.METRIC_CD, 'B1|B2') THEN REGEXP_REPLACE(A.METRIC_CD, 'DB|TB', 'B') ELSE A.METRIC_CD END TMP_METRIC_CD
            , CASE WHEN REGEXP_LIKE(A.METRIC_CD, 'B1|B2') THEN REGEXP_REPLACE(A.METRIC_NAME, ' : DTAC| : TMH') ELSE A.METRIC_NAME END TMP_METRIC_NAME
            , CASE WHEN REGEXP_LIKE(SUBSTR(A.METRIC_CD,-2), 'A[A-Z]$') THEN SUBSTR(A.METRIC_CD,-2) ELSE 'ALL' END CHANNEL_CD
            , A.AREA_NO, A.AREA_TYPE, A.AREA_CD, A.AREA_NAME
            , COALESCE(A.ACTUAL_SNAP, 0) AS ACTUAL_SNAP
            , COALESCE(CASE WHEN T.METRIC_CD IS NOT NULL THEN T.TARGET_MTH
                            ELSE A.TARGET_SNAP
                            END, 0) AS TARGET_SNAP
            , A.PPN_TM, A.LOAD_DATE
        
        FROM AUTOKPI.ADHOC_RAW_SALE_PERFORMANCE_MTH A
        
        LEFT JOIN W_SALE_TARGET_MONTHLY T
            ON A.TM_KEY_MTH = T.TM_KEY_MTH
            AND A.METRIC_CD = T.METRIC_CD
            AND A.AREA_CD = T.AREA_CD
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_TOTAL_AGG AS 
    (	
        -->> ALL = (TRUE + DTAC)
        SELECT A.*
            , T.ACTUAL_SNAP AS T_ACTUAL, T.TARGET_SNAP AS T_TARGET
            , D.ACTUAL_SNAP AS D_ACTUAL, D.TARGET_SNAP AS D_TARGET
        
        FROM (
            SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH
                , SUBSTR(TM_KEY_MTH, 5, 2) AS MTH
                , PRODUCT_GRP
                , 'ALL' AS COMP_CD, TMP_METRIC_CD AS METRIC_CD, TMP_METRIC_NAME AS METRIC_NAME
                , CHANNEL_CD, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
                , SUM(ACTUAL_SNAP) AS ACTUAL, SUM(TARGET_SNAP) AS TARGET
                , MAX(PPN_TM) AS PPN_TM, MAX(LOAD_DATE) AS LOAD_DATE
            FROM W_ADJUST_TARGET_DTAC
            GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, PRODUCT_GRP, TMP_METRIC_CD, TMP_METRIC_NAME, CHANNEL_CD, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
        ) A
        
        LEFT JOIN ( -->> TRUE
            SELECT TM_KEY_MTH, COMP_CD, TMP_METRIC_CD, TMP_METRIC_NAME, CHANNEL_CD, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP
            FROM W_ADJUST_TARGET_DTAC
            WHERE COMP_CD = 'TRUE'
        ) T
            ON A.TM_KEY_MTH = T.TM_KEY_MTH
            AND A.METRIC_CD = T.TMP_METRIC_CD
            AND A.AREA_CD = T.AREA_CD
            
        LEFT JOIN ( -->> DTAC
            SELECT TM_KEY_MTH, COMP_CD, TMP_METRIC_CD, TMP_METRIC_NAME, CHANNEL_CD, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP
            FROM W_ADJUST_TARGET_DTAC
            WHERE COMP_CD = 'DTAC'
        ) D
            ON A.TM_KEY_MTH = D.TM_KEY_MTH
            AND A.METRIC_CD = D.TMP_METRIC_CD
            AND A.AREA_CD = D.AREA_CD
    )
    -----------------------------------------------------------------------------------------------------------------------

    -->> Output

    SELECT TM_KEY_YR AS YR, MTH, PRODUCT_GRP, METRIC_CD, METRIC_NAME, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, ACHIEVEMENT, MOM, YOY
        -->> Sales Performance Color
        , CASE 	WHEN ACTUAL > TARGET THEN 'green'
                WHEN ACTUAL < TARGET AND ACTUAL > LAST_YR_ACTUAL THEN 'yellow'
                WHEN ACTUAL < TARGET AND ACTUAL < LAST_YR_ACTUAL THEN 'red'
                END SALE_PERF_COLOR
        , LAST_YR_ACTUAL, LAST_MTH_ACTUAL, ACTUAL, TARGET, T_ACTUAL, T_TARGET, D_ACTUAL, D_TARGET, LOAD_DATE
        
    FROM (
        SELECT TM_KEY_YR, TM_KEY_MTH, MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, CHANNEL_CD, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
            -->> % Achievement
            , CASE 	WHEN COALESCE(ACTUAL,0) = 0 OR COALESCE(TARGET,0) = 0 THEN NULL 
                    ELSE ACTUAL / TARGET * 100 END ACHIEVEMENT
            -->> % MoM
            , CASE 	WHEN COALESCE(ACTUAL,0) = 0 OR COALESCE(LAST_MTH_ACTUAL,0) = 0 THEN NULL 
                    ELSE (ACTUAL - LAST_MTH_ACTUAL) / LAST_MTH_ACTUAL * 100 END MOM
            -->> % YoY
            , CASE 	WHEN COALESCE(ACTUAL,0) = 0 OR COALESCE(LAST_YR_ACTUAL,0) = 0 THEN NULL 
                    ELSE (ACTUAL - LAST_YR_ACTUAL) / LAST_YR_ACTUAL * 100 END YOY
            , LAST_YR_ACTUAL, LAST_MTH_ACTUAL, ACTUAL, TARGET, T_ACTUAL, T_TARGET, D_ACTUAL, D_TARGET, LOAD_DATE
            
        FROM (
            SELECT TM_KEY_YR, TM_KEY_MTH, MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, CHANNEL_CD, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
                , LAG(ACTUAL) OVER(PARTITION BY METRIC_CD, AREA_CD, MTH ORDER BY TM_KEY_YR) LAST_YR_ACTUAL
                , LAG(ACTUAL) OVER(PARTITION BY METRIC_CD, AREA_CD, TM_KEY_YR ORDER BY TM_KEY_MTH) LAST_MTH_ACTUAL
                , ACTUAL, TARGET, T_ACTUAL, T_TARGET, D_ACTUAL, D_TARGET, LOAD_DATE
            FROM W_TOTAL_AGG
        ) AGG_1
        
    ) AGG_2

    WHERE TM_KEY_MTH BETWEEN 202403 AND 202405

    ORDER BY TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_NO, AREA_CD 
"""


# AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()
op_dir = 'output'
op_file = 'sales_performance.xlsx'


try:
    print(f'\n Processing...')
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=['YR', 'MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACHIEVEMENT', 'MOM', 'YOY', 'SALE_PERF_COLOR', 'LAST_YR_ACTUAL', 'LAST_MTH_ACTUAL', 'ACTUAL', 'TARGET', 'T_ACTUAL', 'T_TARGET', 'D_ACTUAL', 'D_TARGET', 'LOAD_DATE'])
    # print(f'\n{df}')
    df.to_excel(f'{op_dir}/{op_file}', sheet_name='Data', index=False)
    print(f'\n -> Generate "{op_file}" successfully')


except oracledb.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    print(f'\nJob Done !!!')


Job Start...

AKPIPRD : Connected

 Processing...

 -> Generate "sales_performance.xlsx" successfully

AKPIPRD : Disconnected

Job Done !!!
